In [ ]:
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import boto3
import tarfile
import os

In [ ]:
# Download model artifact from S3
s3 = boto3.client('s3')
bucket = 'your-bucket-name'
s3.download_file(bucket, 'output/xgboost-YYYY-MM-DD-HH-MM-SS-XXX/model.tar.gz', 'model.tar.gz')


In [ ]:
# Extract model
with tarfile.open('model.tar.gz', 'r:gz') as tar:
    tar.extractall('model')

In [ ]:
# Load model
model = xgb.Booster()
model.load_model('model/xgboost-model')

In [ ]:
# Load data to get feature names
data = pd.read_csv('data/telco_churn.csv')
data = data.drop(columns=['customerID', 'Churn'])
feature_names = data.columns

In [ ]:
# Get feature importance
importance = model.get_score(importance_type='gain')
importance = {feature_names[int(k.replace('f', ''))]: v for k, v in importance.items()}

In [ ]:
# Plot
plt.figure(figsize=(10, 6))
pd.Series(importance).sort_values().plot(kind='barh')
plt.title('Feature Importance for Churn Prediction')
plt.xlabel('Gain')
plt.savefig('feature_importance.png')
plt.show()